# 多进程、线程、协程、事件驱动及select poll epoll

-多线程使用场景

-多进程

 --简单的一个多进程例子

 --进程间数据的交互实现方法

 　　---通过Queues和Pipe可以实现进程间数据的传递，但是不能实现数据的共享

　　　---Queues

    ---Pipe

    ---通过Manager可以不同进程间实现数据的共享

 --进程同步，即进程锁

 --进程池

-协程

 --先用yield实现简单的协程

 --Greenlet

 --Gevent

 --用协程gevent写一个简单并发爬网页

-事件驱动

 --IO多路复用

   ---用户空间和内核空间

    ---文件描述符fd

    ---缓存IO

　--IO模式

 　　---阻塞I/O（blocking IO）

    ---非阻塞I/O

    ---I/O多路复用（IO multiplexing）

    ---异步I/O（asynchronous IO）

-关于select poll epoll

 --select

 --poll

 --epoll

 --以select方法为例子进行理解

多线程的使用场景
IO操作不占用CPU

计算占用cpu

python多线程不适合cpu密集型操作的任务，适合IO操作密集型的任务

多进程
简单的一个多进程例子：（用于理解对多线程方法的使用）
和线程的方法类似，下面是一个简单的多进程代码

复制代码
 1 #AUTHOR:FAN
 2 import time,multiprocessing
 3 
 4 def run(name):
 5     time.sleep(2)
 6     print("hello",name)
 7 
 8 if __name__ =="__main__":
 9     for i in range(6):
10         p = multiprocessing.Process(target=run,args=("dean",))
11       p.start()
复制代码
和之前学习的多线程结合在一起使用，代码如下：

复制代码
 1 #AUTHOR:FAN
 2 
 3 import time,threading
 4 import multiprocessing
 5 
 6 def thread_run():
 7     print(threading.get_ident())   #这里表示获取线程id
 8 
 9 
10 def run(name):
11     time.sleep(2)
12     print("hello",name)
13     t=threading.Thread(target=thread_run)
14     t.start()
15 
16 if __name__ =="__main__":
17     for i in range(6):
18         p = multiprocessing.Process(target=run,args=("dean",))
19         p.start()
复制代码
运行结果如下：


复制代码
 1 D:\python35\python.exe D:/python培训/s14/day10/进程与线程结合使用.py
 2 hello dean
 3 10008
 4 hello dean
 5 9276
 6 hello dean
 7 8096
 8 hello dean
 9 1308
10 hello dean
11 hello dean
12 10112
13 8032
14 
15 Process finished with exit code 0
复制代码
接着我们查看下面代码：

复制代码
 1 #AUTHOR:FAN
 2 
 3 from multiprocessing import Process
 4 import os
 5 
 6 
 7 def info(title):
 8     print(title)
 9     print('module name:', __name__)
10     print('parent process:', os.getppid())
11     print('process id:', os.getpid())
12     print("\n\n")
13 
14 
15 def f(name):
16     info('\033[31;1mcalled from child process function f\033[0m')
17     print('hello', name)
18 
19 if __name__ == '__main__':
20     info('\033[32;1mmain process line\033[0m')
复制代码
运行结果如下：

D:\python35\python.exe D:/python培训/s14/day10/获取进程id.py
main process line
module name: __main__
parent process: 8368
process id: 7664




Process finished with exit code 0


我们这里可以看到父进程id:8368,并且会发现无论程序运行多少次都是这个，然后我们在windows任务管理器查看发现这个是pycharm的进程id,如下图：

![](images/3.png)

这里要记住：每一个子进程都是由父进程启动的

我们将上面代码中if __name__=”__main__”进行修改，如下：

1 if __name__ == '__main__':
2     info('\033[32;1mmain process line\033[0m')
3     p = Process(target=f, args=('bob',))
4     p.start()

![](images/4.png)

进程间数据的交互,实现方法
通过Queues和Pipe可以实现进程间数据的传递，但是不能实现数据的共享
 

不同进程间内存不是共享的，要想实现两个进程间的数据交换，有一下方法：

Queues
使用方法和threading里的queue使用差不多

先回忆一下线程之间的数据共享，通过下面代码理解：

 

复制代码
 1 #AUTHOR:FAN
 2 import threading
 3 import queue
 4 
 5 def func():
 6     q.put([22,"dean",'hello'])
 7 
 8 if __name__=="__main__":
 9     q = queue.Queue()
10     t = threading.Thread(target=func)
11     t.start()
12     print(q.get(q))
复制代码
 

运行结果：

 View Code
从上述代码可以看出线程之间的数据是共享的：父线程可以访问子线程放入的数据

如果是多进程之间呢？

将代码进行修改如下，让子进程调用父进程数据：

 

复制代码
 1 from multiprocessing import Process
 2 import queue
 3 
 4 
 5 
 6 def f():
 7     q.put([11,None,"hello"])
 8 
 9 
10 if __name__=="__main__":
11     q = queue.Queue()
12     p = Process(target=f)
13     p.start()
14     print(q.get())

![](images/5.png)

事件驱动
通常，我们写服务器处理模型的程序时，有以下几种模型：

（1）每收到一个请求，创建一个新的进程，来处理该请求；

（2）每收到一个请求，创建一个新的线程，来处理该请求；

（3）每收到一个请求，放入一个事件列表，让主进程通过非阻塞I/O方式来处理请求

上面的几种方式，各有千秋，

第（1）中方法，由于创建新的进程的开销比较大，所以，会导致服务器性能比较差,但实现比较简单。

第（2）种方式，由于要涉及到线程的同步，有可能会面临死锁等问题。

第（3）种方式，在写应用程序代码时，逻辑比前面两种都复杂。

综合考虑各方面因素，一般普遍认为第（3）种方式是大多数网络服务器采用的方式

目前大部分的UI编程都是事件驱动模型，如很多UI平台都会提供onClick()事件，这个事件就代表鼠标按下事件。事件驱动模型大体思路如下：

1. 有一个事件（消息）队列；

2. 鼠标按下时，往这个队列中增加一个点击事件（消息）；

3. 有个循环，不断从队列取出事件，根据不同的事件，调用不同的函数，如onClick()、onKeyDown()等；

4. 事件（消息）一般都各自保存各自的处理函数指针，这样，每个消息都有独立的处理函数



 

事件驱动编程是一种编程范式，这里程序的执行流由外部事件来决定。它的特点是包含一个事件循环，当外部事件发生时使用回调机制来触发相应的处理。另外两种常见的编程范式是（单线程）同步以及多线程编程。

让我们用例子来比较和对比一下单线程、多线程以及事件驱动编程模型。下图展示了随着时间的推移，这三种模式下程序所做的工作。这个程序有3个任务需要完成，每个任务都在等待I/O操作时阻塞自身。阻塞在I/O操作上所花费的时间已经用灰色框标示出来了。



在单线程同步模型中，任务按照顺序执行。如果某个任务因为I/O而阻塞，其他所有的任务都必须等待，直到它完成之后它们才能依次执行。这种明确的执行顺序和串行化处理的行为是很容易推断得出的。如果任务之间并没有互相依赖的关系，但仍然需要互相等待的话这就使得程序不必要的降低了运行速度。

 

在多线程版本中，这3个任务分别在独立的线程中执行。这些线程由操作系统来管理，在多处理器系统上可以并行处理，或者在单处理器系统上交错执行。这使得当某个线程阻塞在某个资源的同时其他线程得以继续执行。与完成类似功能的同步程序相比，这种方式更有效率，但程序员必须写代码来保护共享资源，防止其被多个线程同时访问。多线程程序更加难以推断，因为这类程序不得不通过线程同步机制如锁、可重入函数、线程局部存储或者其他机制来处理线程安全问题，如果实现不当就会导致出现微妙且令人痛不欲生的bug。

 

在事件驱动版本的程序中，3个任务交错执行，但仍然在一个单独的线程控制中。当处理I/O或者其他昂贵的操作时，注册一个回调到事件循环中，然后当I/O操作完成时继续执行。回调描述了该如何处理某个事件。事件循环轮询所有的事件，当事件到来时将它们分配给等待处理事件的回调函数。这种方式让程序尽可能的得以执行而不需要用到额外的线程。事件驱动型程序比多线程程序更容易推断出行为，因为程序员不需要关心线程安全问题。

当我们面对如下的环境时，事件驱动模型通常是一个好的选择：

 

（1）程序中有许多任务

（2）任务之间高度独立（因此它们不需要互相通信，或者等待彼此）

（3）在等待事件到来时，某些任务会阻塞。

当应用程序需要在任务间共享可变的数据时，这也是一个不错的选择，因为这里不需要采用同步处理。

网络应用程序通常都有上述这些特点，这使得它们能够很好的契合事件驱动编程模型。

IO多路复用
用户空间和内核空间
操作系统都是采用虚拟存储器，对于32位操作系统，它的寻址空间（虚拟存储空间）为4G。操作系统的核心是内核，独立于普通的应用程序，可以访问受保护内存空间，也有访问底层硬件设备的所有权限，为了保证用户进程不能直接操作内核，保证内核的安全，操作系统将虚拟空间分为两部分：一部分为内核空间，一部分是用户空间，针对linux系统而言，将最高的1G字节给内核使用，称为内核空间，将3G字节的供各个进程使用，称为用户空间

文件描述符fd
文件描述符是一个用于表述指向文件的引用的抽象化概念

文件描述符在形式上是一个非负整数，实际上，它是一个索引值，指内核为每一个进程所维护的进程打开文件的记录的记录表，当程序打开一个现有文件或者创建一个新文件时，内核向进程返回一个文件描述符。

缓存IO
缓存IO，也被称为标准IO，大多数文件系统默认IO操作都是缓存IO，在Linux的缓存IO机制中，操作系统会将IO的数据缓存在文件系统的页缓存（page cache）中，也就是说，数据会先被拷贝到操作系统内核的缓冲区中，然后才会从操作系统内核的缓冲区拷贝到应用程序的地址空间

缓存IO的缺点：

数据在传输过程中需要在应用程序地址空间和内核进行多次数据拷贝操作，这些数据拷贝操作所带来的CPU以及内存开销是非常大的

IO模式
对于一次IO访问（以read为例子），数据会先拷贝到操作系统内核的缓冲区中，然后会从操作系统内核的缓冲区拷贝到应用程序的地址空间，也就是说当一个read操作发生时，它会经历两个阶段：

1. 等待数据准备

2. 经数据从内核拷贝到进程

正是因为这两个阶段，linux系统产生了五种网络模式的方案

1. 阻塞I/O（blocking IO）

2. 非阻塞I/O（nonblocking IO）

3. I/O多路复用（IO multiplexing）

4. 信号驱动I/O（signal driven IO）

5. 异步I/O（asynchromous IO）

注意：信号驱动I/O（signal driven IO）在实际中不常用

阻塞I/O（blocking IO）
在linux中，默认情况下所有的socket都是blocking，一个典型的读操作流程大概是这样：



当用户进程调用了recvfrom这个系统调用，kernel就开始了IO的第一个阶段：准备数据（对于网络IO来说，很多时候数据在一开始还没有到达。比如，还没有收到一个完整的UDP包。这个时候kernel就要等待足够的数据到来）。这个过程需要等待，也就是说数据被拷贝到操作系统内核的缓冲区中是需要一个过程的。而在用户进程这边，整个进程会被阻塞（当然，是进程自己选择的阻塞）。当kernel一直等到数据准备好了，它就会将数据从kernel中拷贝到用户内存，然后kernel返回结果，用户进程才解除block的状态，重新运行起来。

 

所以，blocking IO的特点就是在IO执行的两个阶段都被block了

非阻塞I/O
linux下，可以通过设置socket使其变为non-blocking。当对一个non-blocking socket执行读操作时，流程是这个样子：



当用户进程发出read操作时，如果kernel中的数据还没有准备好，那么它并不会block用户进程，而是立刻返回一个error。从用户进程角度讲 ，它发起一个read操作后，并不需要等待，而是马上就得到了一个结果。用户进程判断结果是一个error时，它就知道数据还没有准备好，于是它可以再次发送read操作。一旦kernel中的数据准备好了，并且又再次收到了用户进程的system call，那么它马上就将数据拷贝到了用户内存，然后返回。

 

所以，nonblocking IO的特点是用户进程需要不断的主动询问kernel数据好了没有。

I/O多路复用（IO multiplexing）
IO multiplexing就是我们说的select，poll，epoll，有些地方也称这种IO方式为event driven IO。select/epoll的好处就在于单个process就可以同时处理多个网络连接的IO。它的基本原理就是select，poll，epoll这个function会不断的轮询所负责的所有socket，当某个socket有数据到达了，就通知用户进程。



 

当用户进程调用了select，那么整个进程会被block，而同时，kernel会“监视”所有select负责的socket，当任何一个socket中的数据准备好了，select就会返回。这个时候用户进程再调用read操作，将数据从kernel拷贝到用户进程。

 

所以，I/O 多路复用的特点是通过一种机制一个进程能同时等待多个文件描述符，而这些文件描述符（套接字描述符）其中的任意一个进入读就绪状态，select()函数就可以返回。

 

这个图和blocking IO的图其实并没有太大的不同，事实上，还更差一些。因为这里需要使用两个system call (select 和 recvfrom)，而blocking IO只调用了一个system call (recvfrom)。但是，用select的优势在于它可以同时处理多个connection。

 

所以，如果处理的连接数不是很高的话，使用select/epoll的web server不一定比使用multi-threading + blocking IO的web server性能更好，可能延迟还更大。select/epoll的优势并不是对于单个连接能处理得更快，而是在于能处理更多的连接。）

 

在IO multiplexing Model中，实际中，对于每一个socket，一般都设置成为non-blocking，但是，如上图所示，整个用户的process其实是一直被block的。只不过process是被select这个函数block，而不是被socket IO给block。

异步I/O（asynchronous IO）
Linux下的asynchronous IO其实用得很少。先看一下它的流程：



 

用户进程发起read操作之后，立刻就可以开始去做其它的事。而另一方面，从kernel的角度，当它受到一个asynchronous read之后，首先它会立刻返回，所以不会对用户进程产生任何block。然后，kernel会等待数据准备完成，然后将数据拷贝到用户内存，当这一切都完成之后，kernel会给用户进程发送一个signal，告诉它read操作完成了。

关于select poll epoll
select
sekect是通过一个select（）系统调用来监视多个文件描述符，当select()返回后，该数组中就绪的文件描述符便会被该内核修改标志位，使得进程可以获得这些文件描述符从而进行后续的读写操作

select的优点就是支持跨平台

缺点在于单个进程能够监视的文件描述符的数量存在最大限制

另外select()所维护的存储大量文件描述符的数据结构，随着文件描述符数量的增大，其复制的开销也线性增长。同时，由于网络响应时间的延迟使得大量TCP连接处于非活跃状态，但调用select()会对所有socket进行一次线性扫描，所以这也浪费了一定的开销。

poll
和select在本质上没有多大差别，但是poll没有最大文件描述符数量的限制

poll和select同样存在一个缺点就是，包含大量文件描述符的数组被整体复制于用户态和内核的地址空间之间，而不论这些文件描述符是否就绪，它的开销随着文件描述符数量的增加而线性增大。

另外，select()和poll()将就绪的文件描述符告诉进程后，如果进程没有对其进行IO操作，那么下次调用select()和poll()的时候将再次报告这些文件描述符，所以它们一般不会丢失就绪的消息，这种方式称为水平触发（Level Triggered）。

 

epoll
epoll可以同时支持水平触发和边缘触发（Edge Triggered，只告诉进程哪些文件描述符刚刚变为就绪状态，它只说一遍，如果我们没有采取行动，那么它将不会再次告知，这种方式称为边缘触发），理论上边缘触发的性能要更高一些，但是代码实现相当复杂。

 

epoll同样只告知那些就绪的文件描述符，而且当我们调用epoll_wait()获得就绪文件描述符时，返回的不是实际的描述符，而是一个代表就绪描述符数量的值，你只需要去epoll指定的一个数组中依次取得相应数量的文件描述符即可，这里也使用了内存映射（mmap）技术，这样便彻底省掉了这些文件描述符在系统调用时复制的开销。

 

另一个本质的改进在于epoll采用基于事件的就绪通知方式。在select/poll中，进程只有在调用一定的方法后，内核才对所有监视的文件描述符进行扫描，而epoll事先通过epoll_ctl()来注册一个文件描述符，一旦基于某个文件描述符就绪时，内核会采用类似callback的回调机制，迅速激活这个文件描述符，当进程调用epoll_wait()时便得到通知

 

以select方法为例子进行理解
Python的select()方法直接调用操作系统的IO接口，它监控sockets,open files, and pipes(所有带fileno()方法的文件句柄)何时变成readable 和writeable, 或者通信错误，select()使得同时监控多个连接变的简单，并且这比写一个长循环来等待和监控多客户端连接要高效，因为select直接通过操作系统提供的C的网络接口进行操作，而不是通过Python的解释器。

接下来通过echo server例子要以了解select 是如何通过单进程实现同时处理多个非阻塞的socket连接的

代码如下：

复制代码
 1 #AUTHOR:FAN
 2 
 3 import select
 4 import socket
 5 import queue
 6 server = socket.socket()
 7 server.bind(('127.0.0.1',9999))
 8 server.listen()
 9 
10 server.setblocking(False)#不阻塞
11 msg_dict = {}
12 inputs=[server,]
13 outputs=[]
14 
15 while True:
16     readable, writeable, exceptional = select.select(inputs, outputs, inputs)
17     print(readable, writeable, exceptional)
18     for r in readable:
19         if r is server:   #代表来了一个新连接
20             conn,addr = server.accept()
21             print("来了一个新连接：",addr)
22             inputs.append(conn)  #是因为这个新建立的连接还没发数据过来，现在就接收的话程序就报错了
23             #所以要想要实现这个客户端发数据来时server端能知道，就需要让select再监测这个conn
24             msg_dict[conn] = queue.Queue() #初始化一个队列，后面需要返回给这个客户端的数据
25         else:
26             data = r.recv(1024)
27             print("收到数据：",data)
28             msg_dict[r].put(data)
29             outputs.append(r)  #放入返回的连接队列里
30 
31     for w in writeable:    #要返回给客户端的连接列表
32         data_to_client = msg_dict[w].get()
33         w.send(data_to_client)  #返回给客户端源数据
34         outputs.remove(w)   #确保下次循环的时候writeable，不能返回这个已经处理完的连接了
35     for e in exceptional:
36         if e in outputs:
37             outputs.remove(e)
38         inputs.remove(e)
39         del msg_dict[e]
复制代码
其实上述的代码相对来说是比较麻烦，python已经封装了selectors模块，并且这个模块中包含了select和epoll,会根据系统自动识别（windows只支持select,linux是二者都支持），默认用epoll

如果将上述代码用selectors模块的方式写，代码如下：

 

复制代码
 1 #AUTHOR:FAN
 2 
 3 
 4 import selectors
 5 import socket
 6 
 7 sel = selectors.DefaultSelector()
 8 def accept(server,mask):
 9     conn,addr = server.accept()
10     print("一个新的连接",addr)
11     print(conn)
12     conn.setblocking(False)
13     sel.register(conn,selectors.EVENT_READ,read)  #新连接注册read回调函数
14     print("done")
15 
16 def read(conn,mask):
17     print("ccc")
18     print("mask:",mask)
19     data = conn.recv(1024)
20     if data:
21         print(data)
22         conn.send(data)
23     else:
24         print("客户端断开连接")
25         sel.unregister(conn)
26         conn.close()
27 
28 server = socket.socket()
29 server.bind(('127.0.0.1',9999))
30 server.listen()
31 server.setblocking(False)
32 sel.register(server,selectors.EVENT_READ,accept)
33 
34 while True:
35     print("cccccccsssssss")
36     events = sel.select() #默认阻塞，有活动连接，有活动连接就返回活动的连接列表
37     print(events)
38     for key,mask in events:
39         print("key:%s    mask:%s"%(key,mask))
40         callback = key.data  #这里就是回调函数及上述的accept
41         print("key.data:",key.data)
42         print("key.fileobj:",key.fileobj)
43         callback(key.fileobj,mask) #key.fileobj
复制代码
 

我们用客户端模拟同时并发一万去连接服务端

客户端代码如下：

复制代码
 1 #AUTHOR:FAN
 2 
 3 
 4 import socket
 5 import sys
 6 
 7 messages = [ b'This is the message. ',
 8              b'It will be sent ',
 9              b'in parts.',
10              ]
11 server_address = ('192.168.8.102', 10000)
12 socks = [ socket.socket(socket.AF_INET, socket.SOCK_STREAM) for i in range(10000)
13           ]
14 print('connecting to %s port %s' % server_address)
15 for s in socks:
16     s.connect(server_address)
17 
18 for message in messages:
19     for s in socks:
20         print('%s: sending "%s"' % (s.getsockname(), message) )
21         s.send(message)
22     for s in socks:
23         data = s.recv(1024)
24         print( '%s: received "%s"' % (s.getsockname(), data) )
25         if not data:
26             print(sys.stderr, 'closing socket', s.getsockname() )
复制代码
将服务端放到linux服务端，在本机执行客户端，从而实现了上万的并发


http://www.cnblogs.com/zhaof/p/5932461.html